In [ ]:
#!/usr/bin/env python

import sys
sys.path.append('/home/bij/Projects/fdtd/')
import fdtd
import fdtd.backend as bd
import matplotlib.pyplot as plt
import numpy as np


# ## Set Backend
#fdtd.set_backend("numpy")
fdtd.set_backend("torch")


# ## Constants
WAVELENGTH = 1550e-9
WAVELENGTH2 = 1550e-8
SPEED_LIGHT: float = 299_792_458.0  # [m/s] speed of light



In [ ]:
for shift in range(0, 14):
    grid = fdtd.Grid(
        (1.5e-5, 1.5e-5, 1),
        grid_spacing=0.1 * WAVELENGTH,
        permittivity=1.0,
        permeability=1.0,
    )
    grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
    grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
    grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
    grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
    grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")
    
    y_mid, x_mid = grid.shape[0]//2, grid.shape[1]//2
    
    grid[y_mid, 25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource0", amplitude=1,
    )
    grid[y_mid, -25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource1", delay=shift,
    )

    grid_E_energies = None
    grid.visualize(z=0, animate=True)
    for i in range(100):
        grid.run(1, progress_bar=False)
        grid_energy_E, grid_energy_H = grid.visualize(z=0, norm='linear', animate=True)
        if(i == 0):
            grid_E_energies = grid_energy_E
        else:
            grid_E_energies += grid_energy_E
        #plt.show()
    plt.imshow(bd.numpy(grid_E_energies))
    plt.savefig('avg_energy_shift_mid={0}.png'.format(shift))


In [ ]:
for shift in range(0, 14):
    shift = 14
    grid = fdtd.Grid(
        (1.5e-5, 1.5e-5, 1),
        grid_spacing=0.1 * WAVELENGTH,
        permittivity=1.0,
        permeability=1.0,
    )
    grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
    grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
    grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
    grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
    grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")
    
    y_mid, x_mid = grid.shape[0]//2, grid.shape[1]//2
    
    grid[y_mid, 25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource0", amplitude=1,
    )
    grid[y_mid, -25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource1", delay=shift,
    )

    
    
    print('Running shift {0}'.format(shift))
    grid.run(1000, progress_bar=False)
    plt.imshow(bd.numpy(grid.E_avg[:, :, 0, :]))
    plt.savefig('avg_energy_shift_mid={0}.png'.format(shift))
    break


In [ ]:
plt.imshow(bd.numpy(bd.sum(grid.E_avg[:, :, 0, :]**2, axis=-1)))

In [ ]:
avg_energy = bd.numpy(bd.sum(grid.E_avg[:, :, 0, :]**2, axis=-1))

In [ ]:
import numpy as np
plt.hist(avg_energy.flatten(), bins=100)

In [ ]:
def NormalizeEnergy(energy, width=3):
    mean = np.mean(energy.flatten())
    std = np.std(energy.flatten())
    h_cutoff = mean + std*width
    energy_normed = (np.clip(energy, 0, h_cutoff))/(h_cutoff)
    return energy_normed

In [ ]:
#plt.hist(avg_energy.flatten(), bins=100)
plt.hist(NormalizeEnergy(avg_energy.flatten()), bins=100)

In [ ]:
NormalizeEnergy(avg_energy.flatten()).min()

In [ ]:
plt.imshow(NormalizeEnergy(avg_energy))

In [ ]:
# Video of waves hitting a head

grid = fdtd.Grid(
    (4.5e-5, 4.5e-5, 1),
    grid_spacing=0.1 * WAVELENGTH,
    permittivity=1.0,
    permeability=1.0,
)
grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")

y_mid, x_mid = grid.shape[0]//2, grid.shape[1]//2

grid[25, 25, 0] = fdtd.PointSource(
    period=WAVELENGTH2 / SPEED_LIGHT, name="linesource0", amplitude=1,
)

grid_E_energies = None
grid.visualize(z=0, animate=True)
for i in range(100):
    grid.run(10, progress_bar=False)
    #grid_energy_E, grid_energy_H = grid.visualize(z=0, norm='log', animate=True)
    
    energy_normed = bd.numpy(NormalizeEnergy(bd.sum(grid.E**2, axis=-1).numpy()))
    plt.axis('off')
    plt.imshow(energy_normed, cmap='PuBuGn')
    #plt.show()
    plt.savefig('../waves/waves_{0:04d}.jpg'.format(i), bbox_inches='tight')


In [ ]:
period_steps = 90
major_step = 0
l_delay = 0
r_delay = period_steps//2

grid = fdtd.Grid(
    (3.5e-5, 3.5e-5, 1),
    #(1.5e-5, 1.5e-5, 1),
    grid_spacing=0.1 * WAVELENGTH,
    permittivity=1.0,
    permeability=1.0,
)
grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")

y_mid, x_mid = grid.shape[0]//2, grid.shape[1]//2

grid[y_mid, 25, 0] = fdtd.PointSource(
    period=WAVELENGTH2 / SPEED_LIGHT, name="left_source", delay=l_delay,
)
grid[y_mid, -25, 0] = fdtd.PointSource(
    period=WAVELENGTH2 / SPEED_LIGHT, name="right_source", delay=r_delay,
)

l_delay += 0
r_delay += 0

print('Step: {0}\tRunning with delays ({1}, {2}) and source period {3}'.format(major_step, l_delay, r_delay, grid.sources[0].period))
dead_period = 24
for i in range(100):
    if(i > dead_period):
        grid.run(10, progress_bar=False)
    E_energy_avg = bd.numpy(bd.sum(grid.E_avg[:, :, 0, :]**2, axis=-1))
    E_energy_avg_normed = NormalizeEnergy(E_energy_avg)
    plt.axis('off')
    if(i > dead_period):
        plt.imshow(E_energy_avg_normed)
   
        #plt.show()
        plt.savefig('../wave_ears/energy_{0:04d}.png'.format(i), bbox_inches='tight')
    print(i)

In [ ]:
from PIL import Image
import os
import cv2

def find_images_in_directory(directory):
    """
    Returns a list of image paths found in the specified directory.
    Supports common image formats: .jpg, .jpeg, .png, .bmp, .gif.
    """
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    return [os.path.join(directory, f) for f in os.listdir(directory) if os.path.splitext(f)[1].lower() in img_extensions]

# def combine_images(img_path1, img_path2, img_path_head):
#     """
#     Combines two images side by side and returns the combined image.
#     """
#     image1 = Image.open(img_path1)
#     image2 = Image.open(img_path2)

#     # Calculate dimensions for the combined image
#     total_width = image1.width + image2.width
#     max_height = max(image1.height, image2.height)

#     # Create a new image with appropriate dimensions
#     new_img = Image.new('RGB', (total_width, max_height))

#     # Paste the two images into the new image
#     new_img.paste(image1, (0, 0))
#     new_img.paste(image2, (image1.width, 0))

#     return new_img

def combine_images(img_path1, img_path2, image_head):
    """
    Combines two images side by side and returns the combined image.
    """
    image1 = Image.open(img_path1)
    image2 = Image.open(img_path2)

    # Calculate dimensions for the combined image
    max_width = max(image1.width, image2.width)
    max_height = max(image1.height, image2.height)
    total_width = 2*max_width

    image1 = image1.resize((max_width, max_height))
    image2 = image2.resize((max_width, max_height))
    image_head = image_head.resize((max_width, max_height))
    # Create a new image with appropriate dimensions
    new_img = Image.new('RGB', (total_width, max_height))

    # Paste the two images into the new image
    new_img.paste(image1, (0, 0))
    new_img.paste(image_head, (0, 0), image_head)
    new_img.paste(image2, (image1.width, 0))

    return new_img

def main(directory1, directory2, output_directory):
    """
    Combines images from two directories and saves them to an output directory.
    """
    # Find images in both directories
    dir1_images = sorted(find_images_in_directory(directory1))
    dir2_images = sorted(find_images_in_directory(directory2))

    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    img_head = Image.open('./head_and_ears_tr.png')
    # Iterate over the list of image paths, combining and saving them
    for i, (img_path1, img_path2) in enumerate(zip(dir1_images, dir2_images), start=1):
        combined_img = combine_images(img_path1, img_path2, img_head)
        output_path = os.path.join(output_directory, 'combined_{0:04d}.jpg'.format(i))
        combined_img.save(output_path)
        print(f'Saved: {output_path}')

if __name__ == '__main__':
    directory1 = '../waves/'
    directory2 = '../wave_ears/'
    output_directory = '../combined_waves_and_ears/'
    main(directory1, directory2, output_directory)